In [2]:
from nariflow import start_initializer

start_initializer().initializer('tape')

In [3]:
import numpy as np
import pandas as pd
#import cupy as cp

from nariflow import Variable
from nariflow import optimizer
from nariflow import GradientTape
#from nariflow import calc_gradient
from nariflow import layer
from nariflow.models import Model
from nariflow import functions as f
from nariflow.core import elementary_function as ef
from nariflow.core import shape_function as sf
from nariflow.utils import unit_test
from nariflow.functions import linalg
from nariflow.functions import math
import numpy as np
import pandas as pd

import tensorflow as tf

import torch

import time

No module named 'cupy'


In [4]:
test_module = unit_test.UnitTest()

In [ ]:
test_module.start_testing()

high_order_test_0  : ok
high_order_test_1  : ok
matrix_order_test_0  : ok
matrix_order_test_1  : ok
matrix_order_test_2  : ok


## CORE_TAPE

In [12]:
class Variable():
    def __init__(self, data):
        self.data = data
        self.generation = 0
        self.grad = None

    def set_generation(self, generation):
        self.generation = generation + 1

    def resetgrad(self):
        self.grad = None

    def shape(self):
        return self.data.shape

    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

In [13]:
class Function():
    def __call__(self, *inputs):
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x

        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        inputs = [as_variable(as_array(x)) for x in inputs]
        x_list = [i.data for i in inputs]
        y_list = self.forward(*x_list)
        if not isinstance(y_list, tuple):
            y_list = (y_list,)

        output_list = [Variable(as_array(y)) for y in y_list]
        generation = np.max([i.generation for i in inputs])
        for output in output_list:
            output.set_generation(generation)
        self.generation = generation

        if 'GRADIENT_NUM' in globals():
            GRADIENT_NUM = globals()['GRADIENT_NUM']
            self.making_gradient_tape(output_list, inputs)

        if len(output_list) > 1:
            return output_list
        else:
            return output_list[0]

    def making_gradient_tape(self, output, inputs):
        for i in output:
            GRADIENT_NUM = globals()['GRADIENT_NUM']
            for j in range(GRADIENT_NUM + 1):
                globals()[f'GRADIENT_TAPE_{j}'][i] = (self, inputs, self.generation)

    def forward(self, x_list):
        raise NotImplementedError()

    def backward(self, gy_list):
        raise NotImplementedError()


In [4]:

class GradientTape():

    def __init__(self):
        if 'GRADIENT_NUM' not in globals():
            globals()['GRADIENT_NUM'] = 0
        else:
            globals()['GRADIENT_NUM'] += 1
        GRADIENT_NUM = globals()['GRADIENT_NUM']
        globals()[f'GRADIENT_TAPE_{GRADIENT_NUM}'] = dict()
        self.gradient_tape = globals()[f'GRADIENT_TAPE_{GRADIENT_NUM}']
        self.gradient_num = globals()['GRADIENT_NUM']

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        if f'GRADIENT_TAPE_{self.gradient_num}' in globals():
            del globals()[f'GRADIENT_TAPE_{self.gradient_num}']
        if 'GRADIENT_NUM' in globals():
            del globals()['GRADIENT_NUM']
        return

    def unlist_inputs(self, x):
        input_list = list()
        if len(np.array(x.data).shape) == 0:
            return x
        else:
            for inp in x.data:
                if isinstance(inp, Variable):
                    input_list.append(inp)
                else:
                    return x
            return input_list

    def unlist(self, x):
        inputs = [self.unlist_inputs(i) for i in x]
        if inputs is not None:
            if isinstance(inputs[0], list):
                inputs = sum(inputs, [])

        return inputs

    def CalcGradient(self, target=None, tapes=None, resetgrad=False):
        if tapes is None:
            tapes = self.gradient_tape
        tapes = dict(sorted(tapes.items(), key=lambda x: x[1][2]))

        if target is not None:
            target_ind = [i for i, j in enumerate([i == target for i in list(tapes)]) if j][0]
            tapes_dict = dict()
            [tapes_dict.update(i) for i in [{i[0]: i[1]} for i in tapes.items()][0:target_ind + 1]]
            tapes = dict(reversed(tapes_dict.items()))
        else:
            tapes = dict(reversed(tapes.items()))

        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x

        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)

        for tape in tapes.items():
            outputs = tape[0]
            generation = tape[1][2]
            inputs = tape[1][1]
            func = tape[1][0]

            if isinstance(outputs, Variable):
                outputs = [outputs]

            for j in outputs:
                if j.grad is None:
                    j.grad = Variable(np.ones_like(j.data))

            gy_list = [output.grad for output in outputs]
            func.input_list = inputs
            gx_list = func.backward(*gy_list)
            if not isinstance(gx_list, tuple):
                gx_list = (gx_list,)
            inputs = self.unlist(inputs)

            for x, gx in zip(inputs, gx_list):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
            if resetgrad:
                self.resetgrads()

    def resetgrads(self):
        tapes = self.gradient_tape
        for tape in tapes.items():
            outputs = tape[0]
            inputs = tape[1][1]
            if isinstance(outputs, Variable):
                outputs = [outputs]
            inputs = self.unlist(inputs)

            for x in inputs:
                x.grad = None

            for x in outputs:
                x.grad = None

    def jacobian(self, target=None, tapes=None, var=None, var_return='Variable'):
        if tapes is None:
            tapes = self.gradient_tape.copy()
        tapes = dict(sorted(tapes.items(), key=lambda x: x[1][2]))

        if target is not None:
            target_ind = [i for i, j in enumerate([i == target for i in list(tapes)]) if j][0]
            tapes_dict = dict()
            [tapes_dict.update(i) for i in [{i[0]: i[1]} for i in tapes.items()][0:target_ind + 1]]
            tapes = dict(reversed(tapes_dict.items()))
        else:
            tapes = dict(reversed(tapes.items()))

        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            if isinstance(x, type(np.array([]))):
                if len(x.shape) == 0:
                    return np.array([[x]])
            return x

        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)

        if len(list(tapes.keys())[0].data.shape) >= 2:
            i_max = list(tapes.keys())[0].data.shape[0]
            j_max = list(tapes.keys())[0].data.shape[1]
        else:
            i_max = 1
            j_max = 1

        jacobian_dict = dict()
        for jacobian_iter_i in range(i_max):
            jacobian_dict_j = dict()
            for jacobian_iter_j in range(j_max):
                temp_dict = dict()
                for tape in tapes.items():
                    outputs = tape[0]
                    generation = tape[1][2]
                    inputs = tape[1][1]
                    func = tape[1][0]

                    if isinstance(outputs, Variable):
                        outputs = [outputs]

                    for j in outputs:
                        if j.grad is None:
                            j.data = as_array(j.data)
                            grad_matrix = np.zeros_like(j.data)
                            grad_matrix[jacobian_iter_i][jacobian_iter_j] = 1
                            j.grad = Variable(grad_matrix)

                    gy_list = [output.grad for output in outputs]
                    func.input_list = inputs
                    gx_list = func.backward(*gy_list)
                    if not isinstance(gx_list, tuple):
                        gx_list = (gx_list,)
                    inputs = self.unlist(inputs)
                    for x, gx in zip(inputs, gx_list):
                        if x.grad is None:
                            x.grad = gx
                        else:
                            x.grad = x.grad + gx
                        temp_dict[x] = x.grad.data
                self.resetgrads()
                jacobian_dict_j[jacobian_iter_j] = temp_dict
            jacobian_dict[jacobian_iter_i] = jacobian_dict_j
        if var is None:
            return jacobian_dict

        selected_jacobian = list()
        for i in jacobian_dict:
            for j in jacobian_dict[i]:
                selected_jacobian.append(jacobian_dict[i][j][var])
        if var_return == 'numpy':
            return np.array(selected_jacobian)
        if var_return == 'list':
            return selected_jacobian
        if var_return == 'Variable':
            return [as_variable(x) for x in selected_jacobian]
        else:
            raise Exception('var_return only accpet "numpy", "list" or "Variable"')


# 함수

## 미분 기본 공식

In [5]:
from nariflow.thirdparty.functions import reshape_sum_backward
import numpy as np

# 덧셈
class Add(Function):
    # 정전파 : 두 변수를 더한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 + x_1
        return y

    # 역전파 : 뒷 단계에서 들어온 그레디언트를 양쪽으로 균등하게 흘려보낸다.
    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = sumto(gx0, self.x_0_shape)
            gx1 = sumto(gx1, self.x_1_shape)
        return gx0, gx1


# 곱셈
class Mul(Function):
    # 정전파 : 두 변수를 곱한다.
    def forward(self, x_0, x_1):
        y = x_0 * x_1
        return y

    # 역전파 : 방향을 스위치해서 뒷 단계의 그레디언트와 입력 변수를 곱해 흘려보낸다.
    def backward(self, gy):
        x_0 = self.input_list[0]
        x_1 = self.input_list[1]
        self.x_0_shape = x_0.data.shape
        self.x_1_shape = x_1.data.shape
        gx0, gx1 = gy, gy
        x0 = x_1 * gx1
        x1 = x_0 * gx0
        if self.x_0_shape != self.x_1_shape:
            x0 = sumto(x0, self.x_0_shape)
            x1 = sumto(x1, self.x_1_shape)
        return x0, x1

    # 음수 변환


class Neg(Function):
    # 정전파 : 음수로 바꾼다.
    def forward(self, x):
        return -x

    # 역전파 : 음수로 바꿔 흘려보낸다.
    def backward(self, gy):
        return -gy


# 뺄셈
class Sub(Function):
    # 정전파 : 두 변수를 뺀다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 - x_1
        return y

    # 역전파 : 앞 변수는 그레디언트를, 뒤 변수는 그레디언트 음수를 흘려보낸다.
    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = sumto(gx0, self.x_0_shape)
            gx1 = sumto(gx1, self.x_1_shape)
        return gx0, -gx1


# 나눗셈
class Div(Function):
    # 정전파 : 변수간 나눗셈을 구한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 / x_1
        return y

    # 역전파 : 앞 변수의 경우 1 / a를, 뒤 변수의 경우 (- a / b **2)를 그레디언트와 곱해 흘려보낸다.
    def backward(self, gy):
        x_0, x_1 = self.input_list
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = sumto(gx0, self.x_0_shape)
            gx1 = sumto(gx1, self.x_1_shape)
        gx_0 = (1 / x_1) * gx0
        gx_1 = (- x_0 / (x_1) ** 2) * gx1
        return gx_0, gx_1

    # 거듭제곱


class Pow(Function):
    # Function 클래스에 거듭제곱 수를 init으로 정의한다.
    def __init__(self, power):
        self.power = power

    # 정전파 : 변수에 거듭제곱을 한다.
    def forward(self, x):
        y = x ** self.power
        return y

    # 역전파 : power * x ^ (power - 1) 에 그레디언트를 곱해 흘려보낸다.
    def backward(self, gy):
        x = self.input_list[0]
        gx = self.power * x ** (self.power - 1) * gy
        return gx


def add(x_0, x_1):
    return Add()(x_0, x_1)


def mul(x_0, x_1):
    return Mul()(x_0, x_1)


def neg(x):
    return Neg()(x)


def sub(x_0, x_1):
    return Sub()(x_0, x_1)


def rsub(x_0, x_1):
    return Sub()(x_1, x_0)


def div(x_0, x_1):
    return Div()(x_0, x_1)


def rdiv(x_0, x_1):
    return Div()(x_1, x_0)


def power(x, power):
    return Pow(power)(x)

class Sum(Function):
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims
        self.x_shape = None

    def forward(self, x):
        self.x_shape = x.shape
        y = np.sum(x, axis=self.axis, keepdims=self.keepdims)
        return y

    def backward(self, gy):
        gy, shape = reshape_sum_backward(gy,
                              x_shape=self.x_shape,
                              axis=self.axis,
                              keepdims=self.keepdims)
        gy = reshape(gy, shape)
        gx = broadcast_to(gy, self.x_shape)
        return gx

class MatMul(Function):
    def forward(self, x, w):
        y = x.dot(w)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        w = self.input_list[1]
        gw = matmul(transpose(x), gy)
        gx = matmul(gy, transpose(w))
        return gx, gw

def flowsum(x, axis = None, keepdims = False):
    return Sum(axis, keepdims)(x)

def matmul(x, w):
    return MatMul()(x, w)

class Parameter(Variable):
    pass


# 연산 기본 메소드들을 덮어씌워
# Variable과 연관된 연산은 기호(+, -, *, /)만 사용해도
# 우리가 정의한 연산을 수행하도록 대치한다.
def setup_variable():
    Variable.__add__ = add
    Variable.__radd__ = add
    Variable.__mul__ = mul
    Variable.__rmul__ = mul
    Variable.__neg__ = neg
    Variable.__sub__ = sub
    Variable.__rsub__ = rsub
    Variable.__truediv__ = div
    Variable.__rtruediv__ = rdiv
    Variable.__pow__ = power
    Variable.__getitem__ = get_item

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = exp(x) * gy
        return gx

class Log(Function):
    def forward(self, x):
        y = np.log(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = (1 / x) * gy
        return gx

def exp(x):
    return Exp()(x)

def log(x):
    return Log()(x)

class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = cos(x) * gy
        return gx

def sin(x):
    return Sin()(x)

class Cos(Function):
    def forward(self, x):
        y = np.cos(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        return -sin(x) * gy

def cos(x):
    return Cos()(x)

class StopGradient(Function):
    def forward(self, x):
        return x

    def backward(self, gy):
        return 0

def stop_gradient(x):
    return StopGradient()(x)

## 텐서 형상 함수

In [7]:

from nariflow.thirdparty.functions import sum_to
import numpy as np


class Reshape(Function):
    def __init__(self, shape):
        # 변환을 원하는 모양을 지정한다.
        self.shape = shape

    def forward(self, x):
        # 원본 모양을 저장해준다.
        self.x_shape = x.shape
        # 모양을 reshpae로 바꾼다.
        y = x.reshape(self.shape)
        return y

    def backward(self, gy):
        # 역전파시에는 원본 모양을 복원한다.
        gx = reshape(gy, self.x_shape)
        return gx


def reshape(x, shape):
    return Reshape(shape)(x)


class Transpose(Function):
    def __init__(self, shape=None):
        self.shape = shape

    def forward(self, x):
        # 원본 텐서의 모양을 저장해둔다.
        # transpose를 실시한다.
        y = np.transpose(x, self.shape)
        return y

    def backward(self, gy):
        if self.shape is None:
            gx = transpose(gy)
        # Variable 변수를 받으므로, np.transpose가 아닌 GoteoFlow의 transpose를 받는다.
        # 저장되어있던 원본 모양을 복원한다.
        else:
            axes_len = len(self.shape)
            inv_axes = tuple(np.argsort([ax % axes_len for ax in self.shape]))
            gx = transpose(gy, shape=inv_axes)
        return gx

def transpose(x, shape=None):
    return Transpose(shape)(x)

class SumTo(Function):
    def __init__(self, shape):
        # 모양을 바꾸면서 덧연산을 실시할 목표 모양을 지정한다.
        self.shape = shape

    def forward(self, x):
        # 원본 모양을 기억한다.
        # sum_to 함수로 모양을 바꾸며 합을 실시한다.
        y = sum_to(x, self.shape)
        return y

    def backward(self, gy):
        self.x_shape = self.input_list[0].data.shape
        # 역전파시엔 sum_to로 인해 바뀌었던 모양을 원본 모양으로 복원한다.
        gx = broadcast_to(gy, self.x_shape)
        return gx


def sumto(x, shape):
    return SumTo(shape)(x)


class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = sumto(x, self.x_shape)
        return gx


def broadcast_to(x, shape):
    return BroadcastTo(shape)(x)

class GetItem(Function):
    def __init__(self, slices):
        self.slices = slices

    # 정전파 : 슬라이싱을 수행한다.
    def forward(self, x):
        y = x[self.slices]
        return y

    # 역전파 : 입력 크기와 슬라이싱 정보를 역함수인 GetItemGrad에 전달한다.
    def backward(self, gy):
        x = self.input_list[0]
        f = GetItemGrad(self.slices, x.data.shape)
        return f(gy)

def get_item(x, slices):
    return GetItem(slices)(x)

#GetItemGrad는 GetItem의 역함수다.
class GetItemGrad(Function):
    def __init__(self, slices, shape):
        self.slices = slices
        self.shape = shape

    #정전파(슬라이싱의 역전파) : 입력 크기만큼의 0행렬을 생성한 후, 슬라이싱 위치에 gy를 채워 반환한다.
    # 슬라이싱에서 잘린 성분은 0 그대로 남는다.
    def forward(self, gy):
        gx = np.zeros(self.shape)
        np.add.at(gx, self.slices, gy)
        return gx

    # 역전파(슬라이싱) : 슬라이싱을 수행한다.
    def backward(self, ggx):
        return get_item(ggx, self.slices)

In [8]:
setup_variable()

In [9]:
class DiagonalMat(Function):
    def forward(self, x):
        self.x_shape = x.shape
        y = np.diag(x)
        return y
   
    def backward(self, gy):
        
        gx = diagmat(gy)
        return gx

In [10]:
def diagmat(x):
    return DiagonalMat()(x)

In [11]:
a = Variable(np.array([[3.0,2.0,3.0]]))
b = Variable(np.array([[3.0,2.0,3.0]]))
c = Variable(np.array([[1.0,2.0,3.0]]))

In [12]:
class Concatenate(Function):
    def __init__(self, axis = None):
        if axis is None:
            self.axis = 0
        else:
            self.axis = axis
        
    def forward(self, x):
        x = [i.data for i in x]
        y = np.concatenate(x, axis = self.axis)
        return y
    
    def backward(self, gy):
        x = self.input_list[0]
        shapes = [len(i) for i in x.data]
        concats = list()
        for i in shapes:
            temp = gy[:i]          
            gy = gy[i:]
            concats.append(temp)
        return tuple(concats)
    

In [13]:
class Stack(Function):
    def __init__(self, axis = None):
        if axis is None:
            self.axis = 0
        else:
            self.axis = axis
    
    def forward(self, x):
        x = [i.data for i in x]
        y = np.stack(x, axis = self.axis)
        return y
    
    def backward(self, gy):
        x = self.input_list[0]
        shapes = gy.shape()
        concats = list()
        for axis in range(len(x)):
            ind = ''.join(([':,' if i != self.axis else f'{axis},' for i,j in enumerate(shapes)]))
            temp = eval(f'gy[{ind[:-1]}]')
            concats.append(temp)
        return tuple(concats)

In [14]:
def flowconcat(x, axis = None):
    return Concatenate(axis)(x)

def flowstack(x, axis = None):
    return Stack(axis)(x)

In [240]:
a = Variable(np.array([[[3.0,2.0,1.0]],[[6.,5.,4.]]]))
b = Variable(np.array([[[1.0,2.0,3.0]],[[4.,5.,6.]]]))
c = Variable(np.array([[[1.0,2.0,5.0]],[[7.,8.,9.]]]))

In [241]:
with GradientTape() as tape:
    d = a * b
    concats = linalg.flowconcat([c,d])
    result = concats * concats

In [242]:
tape.CalcGradient()

In [243]:
concats.grad.data

array([[[ 2.,  4., 10.]],

       [[14., 16., 18.]],

       [[ 6.,  8.,  6.]],

       [[48., 50., 48.]]])

In [244]:
c.grad.data

array([[[ 2.,  4., 10.]],

       [[14., 16., 18.]]])

In [245]:
d.grad.data

array([[[ 6.,  8.,  6.]],

       [[48., 50., 48.]]])

In [246]:
a.grad.data

array([[[  6.,  16.,  18.]],

       [[192., 250., 288.]]])

In [247]:
b.grad.data

array([[[ 18.,  16.,   6.]],

       [[288., 250., 192.]]])

In [13]:
with GradientTape() as tape:
    d = a * b
    concats = linalg.flowstack([c,d])
    result = concats * concats

In [14]:
tape.CalcGradient()

In [15]:
result.grad.data

array([[[[1., 1., 1.]],

        [[1., 1., 1.]]],


       [[[1., 1., 1.]],

        [[1., 1., 1.]]]])

In [16]:
concats.grad.data

array([[[[ 2.,  4., 10.]],

        [[14., 16., 18.]]],


       [[[ 6.,  8.,  6.]],

        [[48., 50., 48.]]]])

In [17]:
c.grad.data

array([[[ 2.,  4., 10.]],

       [[14., 16., 18.]]])

In [18]:
d.grad.data

array([[[ 6.,  8.,  6.]],

       [[48., 50., 48.]]])

In [19]:
a.grad.data

array([[[  6.,  16.,  18.]],

       [[192., 250., 288.]]])

In [20]:
b.grad.data

array([[[ 18.,  16.,   6.]],

       [[288., 250., 192.]]])

In [21]:
a = Variable(np.array([[3.0,2.0,1.0],[6.,5.,4.]]))
b = Variable(np.array([[1.0,2.0,3.0],[4.,5.,6.]]))
c = Variable(np.array([[[1.0,2.0,5.0]],[[7.,8.,9.]]]))

In [10]:
class Outer(Function):
    
    def forward(self, x1, x2):
        y = []
        if len(x1.shape) > 2 | len(x2.shape) > 2:
            raise Excetion('Variable must not exceed tensor dimension over 2')
        for i,j in zip(x1, x2):
            y.append(np.outer(i, j))
        return np.array(y)
    
    def backward(self, gy):
        x1, x2 = self.input_list
        main_axis_i = x1.shape()[-1]
        main_axis_j = x2.shape()[-1]
        stack_var_x1 = Variable(np.array([]).reshape(0, main_axis_i))
        stack_var_x2 = Variable(np.array([]).reshape(0, main_axis_j))
        
        for i,j in zip(x1, x2):
            i = reshape(i, [-1,1])
            j = reshape(j, [1,-1])
            
            i = i * main_axis_j
            j = j * main_axis_i
            
            gy_i = sumto(gy, (main_axis_i, 1))
            gy_j = sumto(gy, (1, main_axis_j))
            
            i = i * gy_i
            j = j * gy_j
            i = reshape(i, [1, -1])
            j = reshape(j, [1, -1])
            
            stack_var_x1 = flowconcat([stack_var_x1, i])
            stack_var_x2 = flowconcat([stack_var_x2, j])
                   
        return stack_var_x1, stack_var_x2
    
def outer(x_1, x_2):
    return Outer()(x_1, x_2)

In [56]:
a = Variable(np.array([[[3.0,2.0,1.0]],[[6.,5.,4.]]]))
b = Variable(np.array([[[1.0,2.0,3]],[[4.,5.,6.]]]))

In [57]:
#from nariflow.core.core import Function
#from nariflow.core.core import Variable

In [58]:
with GradientTape() as tape:
    result = linalg.flowconcat([a,b])

In [59]:
result.data

array([[[3., 2., 1.]],

       [[6., 5., 4.]],

       [[1., 2., 3.]],

       [[4., 5., 6.]]])

In [60]:
len(a.data.shape)

3

In [61]:
tape.gradient_tape

{<nariflow.core.core_tape.Variable at 0x2c5cebfde50>: (<nariflow.functions.linalg.Concatenate at 0x2c5cebfd670>,
  0)}

In [62]:
with GradientTape() as tape_2:
    tape.CalcGradient()

In [63]:
b.grad.data

array([[[1., 1., 1.]],

       [[1., 1., 1.]]])

In [64]:
a.grad.data

array([[[1., 1., 1.]],

       [[1., 1., 1.]]])

In [65]:
tape_2.gradient_tape

{<nariflow.core.core_tape.Variable at 0x2c5ceb95430>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95520>,
  0),
 <nariflow.core.core_tape.Variable at 0x2c5ceb95220>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95b50>,
  0),
 <nariflow.core.core_tape.Variable at 0x2c5ceb95a90>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95100>,
  1),
 <nariflow.core.core_tape.Variable at 0x2c5ceb95280>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95a60>,
  1),
 <nariflow.core.core_tape.Variable at 0x2c5ceb95670>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95610>,
  0),
 <nariflow.core.core_tape.Variable at 0x2c5cebfd880>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95910>,
  0),
 <nariflow.core.core_tape.Variable at 0x2c5cebfd3a0>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95ac0>,
  1),
 <nariflow.core.core_tape.Variable at 0x2c5cebfd730>: (<nariflow.core.shape_function.GetItem at 0x2c5ceb95040>,
  2),
 <nariflow.core.core_tape.Variable at 0x2c5cebfdbe0>: (<

In [66]:
tape.resetgrads()

In [67]:
tape_2.CalcGradient()

In [68]:
a.grad.data

array([[[1., 1., 1.]],

       [[1., 1., 1.]]])

In [69]:
b.grad.data

array([[[1., 1., 1.]],

       [[1., 1., 1.]]])

In [379]:
from nariflow.core.core import Variable, Function

In [18]:
class EigenVec(Function):
    
    def forward(self, x):
        self.eigval, self.eigvec = np.linalg.eig(x)
        return self.eigvec
    
    def backward(self, gy):
        x = self.input_list[0]
        second_term = diagmat(self.eigval) - x
        second_term = transpose(second_term)
        geigvec = matmul(self.eigvec, second_term)
        return geigvec * gy

In [19]:
class EigenVal(Function):
    def forward(self, x):
        eigval, eigvec = np.linalg.eig(x)
        self.eigval, self.eigvec = eigval, eigvec
        return eigval
    
    def backward(self, gy):
        eigvec = Variable(self.eigvec)
        geigval = linalg.outer(eigvec, eigvec)
        gy = broadcast_to(gy, eigvec.shape())
        print(geigval, gy)
        return geigval * gy

In [2]:
X = Variable(np.array([[3.0,3.0,2.0],[1.,5.,4.], [2.,3.,4.]]))

In [21]:
class Symmetric(Function):
    
    def forward(self, x):
        y = (1 / 2) * (x + np.transpose(x))
        return y
    
    def backward(self, gy):
        return gy

In [22]:
class TriangleUpper(Function):
    def __init__(self, k):
        if k is not None:
            self.k = k
        else :
            self.k = 0
        
    def forward(self, x):
        y = np.triu(x, k = self.k)
        return y
    
    def backward(self, gy):
        return gy

In [23]:
class TriangleLow(Function):
    def __init__(self, k):
        if k is not None:
            self.k = k
        else :
            self.k = 0
    
    def forward(self, x):
        y = np.tril(x, k = self.k)
        return y
    
    def backward(self, gy):
        return gy
    
def trilower(x):
    return TriangleLow()(x)

In [24]:
class CopyLowtoUpper(Function):
    def forward(self, x):
        y = np.tril(x)
        y_lower = np.tril(x, k = -1)
        y = y + np.transpose(y_lower)
        return y
        
    def backward(self, gy):
        return gy
    
def copyltu(x):
    return CopyLowtoUpper()(x)

In [5]:
class MatrixInv(Function):
    def forward(self, x):
        self.cholesky = np.linalg.cholesky(x)
        y = np.linalg.inv(self.cholesky)
        return y
    
    def backward(self, gy):
        x = self.input_list[0]
        Lt = transpose(matinv(self.cholesky))
        gx = -2 * trilower(matmul(x, matmul(linalg.symmetric(gy), Lt)))
        return gx
    
def matinv(x):
    return MatrixInv()(x)

In [2]:
class CholeskyDecomp(Function):
    def forward(self, x):
        y = np.linalg.cholesky(x)
        self.cholesky = y
        return y
    
    def backward(self, gy):
        cholesky = Variable(self.cholesky)
        cholesky_transpose = transpose(cholesky)
        cholesky_inverse = matinv(cholesky)
        cholesky_inverse_transpose = transpose(cholesky_inverse)
        gx = (1/2) * matmul(cholesky_inverse_transpose,
                       matmul(
                           matmul(copyltu(matmul(cholesky_transpose, gy)),
                                  cholesky_inverse)))
        return gx
    
def cholesky(x):
    return CholeskyDecomp()(x)

NameError: name 'Function' is not defined

In [7]:
class LQDecomp(Function):
    def forward(self, x):
        Q, L = np.linalg.LQDecomp(x)
        self.Q = Q
        self.L = L
        return Q, L
    
    def backward(self, gy):
        gx_Q = gy[0]
        gx_L = gy[1]
        L = Variable(self.L)
        Q = Variable(self.Q)
        L_transpose = transpose(L)
        L_transpose_inverse = matinv(L_transpose)
        Q_transpose = transpose(Q)
        M = matmul(L_transpose, gx_L) - matmul(gx_Q, Q_transpose)
        gx = matmul(L_transpose_inverse, gx_Q + matmul(copyltu(M), Q))
        return gx
        
def lqdecomp(x):
    return LQDecomp()(x)

NameError: name 'Function' is not defined

In [ ]:
class Sign(Function):
    def forward(self, x):
        y = np.sign(x)
        return y
    
    def backward(self, gy):
        return 0
    
def sign(x):
    return Sign()(x)

In [391]:
class Max(Function):
    def __init__(self, axis):
        self.axis = axis
        
    def forward(self, x):
        y = np.max(x, axis = self.axis)
        return y
    
    def backward(self, gy):
        x = self.input_list
        argmax = np.argmax(x, axis = self.axis).reshape(-1)
        shapes = x.shape
        
        gx = Variable(np.zeros(shapes))
        
        shape_list = [list(range(0,i)) for i in shapes]
        shape_list.pop(self.axis)
        index_comb = np.array(np.meshgrid(*shape_list)).T.reshape(-1,len(shape_list))
        result_list = []
        for i,j in zip(index_comb, argmax):
            i = i.tolist()
            i.insert(self.axis, j)
            result_list.append(i)
        result_list = [[str(j) for j in i] for i in result_list]
        result_list = [','.join(i) for i in result_list]
        for i in result_list:
            exec(f'gx[{i}] = gy[{i}]')
        
        return tuple(gx)

In [186]:
class SVDecomp(Function):
    def hfunc(self, x, eta = 1e-8):
        eta_mat = np.broadcast_to(np.array(eta), shape = x.shape())
        return flowmax([flowabs(x).data, eta_mat], axis = 0) * (sign(x) + eta)
    
    def forward(self, x):
        U,S,V = np.linalg.svd(x)
        self.U = U
        self.S = S
        self.V = V
        
    def backward(self, gy):
        gx_U = gy[0]
        gx_S = gy[1]
        gx_V = gy[2]
        U = Variable(self.U)
        S = Variable(self.S)
        V = Variable(self.V)
        
        G_1 = matmul(gx_U, transpose(U)) + (
            matmul(gx_U, transpose(U)) + matmul(matinv(S), matmul(gx_V, matmul(transpose(V), S))))
        
        indicate = Variable(np.ones(G_1.shape)) - Variable(np.identity(G_1.shape[0]))
        h = broadcast_to(S, shape = (S.shape()[-1], S.shape()[-1]))
        E = indicate / (self.hfunc(h - reshape(S, [-1,1])) * self.hfunc(h + reshape(S, [-1,1])))
        
        G_2_b = matmul(matinv(S), matmul(gx_S, transpose(V)))
        identity = diagonal(diagonal(G_2_b))
        G_2 = gx_v + 2 * matmul(symmetric(G_1 * E), S) - identity  
        
        gx = matmul(tranpose(U), matmul(G_2, V) + matmul(matinv(S), gx_V))
        return gx

NameError: name 'Function' is not defined

In [ ]:
class Norm(Function):
    def __init__(self, ord):
        self.ord = ord
        
    def forward(self, x):
        y = np.linalg.norm(x, ord = self.ord)
        self.y = y
        return y
    
    def backward(self, gy):
        x = self.input_list[0]
        gx = ((math.flowabs(x) / self.y) ** (self.ord - 1)) * math.sign(x) * gy
        return gx
    
def norm(x, ord = 2):
    return Norm(ord)(x)

In [4]:
a = Variable(np.array([[3.0,2.0,1.0],[6.,5.,4.]]))

In [5]:
with GradientTape() as tape:
    U,S,V = linalg.svd(a)
    result = S ** 2

In [6]:
tape.gradient_tape

{<nariflow.core.core_tape.Variable at 0x2a3feb91100>: (<nariflow.functions.linalg.SVDecomp at 0x2a3feb91190>,
  0),
 <nariflow.core.core_tape.Variable at 0x2a3feb91e80>: (<nariflow.core.shape_function.GetItem at 0x2a3feb914c0>,
  1),
 <nariflow.core.core_tape.Variable at 0x2a3feb911c0>: (<nariflow.core.shape_function.GetItem at 0x2a3feb91340>,
  1),
 <nariflow.core.core_tape.Variable at 0x2a3feb914f0>: (<nariflow.core.shape_function.GetItem at 0x2a3feb91f70>,
  1),
 <nariflow.core.core_tape.Variable at 0x2a3feb91760>: (<nariflow.core.elementary_function.Pow at 0x2a3feb913a0>,
  2)}

In [7]:
tape.CalcGradient()

In [74]:
a.grad.data

array([[ 3.41058683,  1.92946804,  0.44834925],
       [13.0491406 , 10.87405177,  8.69896293]])

In [9]:
tape.resetgrads()

In [10]:
a = Variable(np.array([[3.0,2.0,1.0],[6.,5.,4.],[1,5,7]]))

In [11]:
with GradientTape() as tape:
    norm = linalg.norm(a)

In [12]:
tape.CalcGradient()

In [84]:
a.grad.data

array([[0.24881074, 0.16587383, 0.08293691],
       [0.49762148, 0.41468457, 0.33174765],
       [0.08293691, 0.41468457, 0.58055839]])

In [34]:
def eigvec(x):
    return EigenVec()(x)

def eigval(x):
    return EigenVal()(x)

In [14]:
a = Variable(np.array([[3.0,2.0,1.0],[6.,5.,4.],[1,5,7]]))

In [15]:
with GradientTape() as tape:
    eig_vec = linalg.eigval(a)

In [16]:
eig_vec.data

array([11.71161347,  3.43742596, -0.14903943])

In [17]:
tape.CalcGradient()

In [97]:
a.grad.data

array([[[ 0.05351613,  0.09193855, -0.07916903],
        [ 0.09193855,  0.15794673, -0.13600921],
        [-0.07916903, -0.13600921,  0.11711862]],

       [[ 0.4125464 ,  0.30730143,  0.50856819],
        [ 0.30730143,  0.22890557,  0.37882705],
        [ 0.50856819,  0.37882705,  0.62693944]],

       [[ 0.53393747, -0.57217351, -0.36967146],
        [-0.57217351,  0.61314769,  0.39614418],
        [-0.36967146,  0.39614418,  0.25594194]]])

In [98]:
tape.resetgrads()

In [226]:
X = Variable(np.array([[2.0,-1.0,0.0],[-1.,2.,-1.], [0.,-1.,2.]]))

In [228]:
with GradientTape() as tape:
    cholesky = linalg.cholesky(X)

In [229]:
tape.CalcGradient()

In [230]:
X.grad.data

array([[2.0115802 , 2.61015892, 1.96820298],
       [2.61015892, 3.39431488, 2.38052044],
       [1.96820298, 2.38052044, 1.48241881]])

In [231]:
tape.resetgrads()

In [232]:
with GradientTape() as tape:
    eig_vec = linalg.eigvec(X)

In [233]:
tape.CalcGradient()

In [234]:
X.grad.data

array([[[-7.07106781e-01, -5.00000000e-01, -0.00000000e+00],
        [-1.00000000e+00, -7.07106781e-01, -0.00000000e+00],
        [ 7.07106781e-01,  5.00000000e-01,  0.00000000e+00]],

       [[-5.00000000e-01,  1.11022302e-16, -5.00000000e-01],
        [-7.07106781e-01,  1.57009246e-16, -7.07106781e-01],
        [ 5.00000000e-01, -1.11022302e-16,  5.00000000e-01]],

       [[-0.00000000e+00, -5.00000000e-01,  7.07106781e-01],
        [-0.00000000e+00, -7.07106781e-01,  1.00000000e+00],
        [ 0.00000000e+00,  5.00000000e-01, -7.07106781e-01]],

       [[ 1.00000000e+00,  7.07106781e-01,  0.00000000e+00],
        [ 5.73329860e-16,  4.05405432e-16,  0.00000000e+00],
        [ 1.00000000e+00,  7.07106781e-01,  0.00000000e+00]],

       [[ 7.07106781e-01, -1.57009246e-16,  7.07106781e-01],
        [ 4.05405432e-16, -9.00180890e-32,  4.05405432e-16],
        [ 7.07106781e-01, -1.57009246e-16,  7.07106781e-01]],

       [[ 0.00000000e+00,  7.07106781e-01, -1.00000000e+00],
        [ 0.00

In [235]:
tape.resetgrads()

In [237]:
with GradientTape() as tape:
    eig_val = linalg.eigval(X)

In [238]:
tape.CalcGradient()

In [239]:
X.grad.data

array([[[ 2.50000000e-01,  3.53553391e-01, -2.50000000e-01],
        [ 3.53553391e-01,  5.00000000e-01, -3.53553391e-01],
        [-2.50000000e-01, -3.53553391e-01,  2.50000000e-01]],

       [[ 5.00000000e-01,  2.86664930e-16,  5.00000000e-01],
        [ 2.86664930e-16,  1.64353564e-31,  2.86664930e-16],
        [ 5.00000000e-01,  2.86664930e-16,  5.00000000e-01]],

       [[ 2.50000000e-01, -3.53553391e-01, -2.50000000e-01],
        [-3.53553391e-01,  5.00000000e-01,  3.53553391e-01],
        [-2.50000000e-01,  3.53553391e-01,  2.50000000e-01]]])

In [12]:
import torch

In [85]:
matrix_torch = torch.autograd.Variable(torch.from_numpy(a.data), requires_grad=True)

In [86]:
result = torch.linalg.norm(matrix_torch)

In [87]:
result.backward()

In [88]:
matrix_torch.grad.data

tensor([[0.2328, 0.1552, 0.0776],
        [0.4657, 0.3881, 0.3105],
        [0.0776, 0.3881, 0.5433]], dtype=torch.float64)

In [89]:
a.grad.data

array([[0.24881074, 0.16587383, 0.08293691],
       [0.49762148, 0.41468457, 0.33174765],
       [0.08293691, 0.41468457, 0.58055839]])

## 테스트 함수

In [336]:
class TestFunction():
    def matyas(self, x, y):
        z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
        return z

    def goldstein(self, x,y):
        z = (1 + (x + y + 1) ** 2 * (19 - 14 * x + 3 * x **2 - 14 * y + 6 * x * y + 3 * y ** 2)) * \
        (30 + (2 * x - 3 * y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
        return z
    
class OrderFunction():
    def high_order_function(self, x, y):
        z = x ** 4 + y ** 3 + x ** 2 + y ** (1/2)
        return z
    def matrix_order_function(self, x):
        y = 2 * x ** 3
        return y
    
class JacobianFunction():
    def matmul(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
        return tape
    
    def reduce_sum(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = ef.flowsum(result_2)
        return tape
    
    def div(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 / result
        return tape
    
    def sum(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 + result
        return tape
    
    def mul(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 * result
        return tape

class GradientStartFunction():
    def gradient_start_middle(self, x, y, k):
        x = Variable(np.array([[1.,2.],[4.,5.]]))
        v = Variable(np.array([[4.,5.],[6.,7.]]))
        k = Variable(np.array([[1.,3.],[4.,6.]]))
        
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(v))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 / x
        
        tape.CalcGradient(target = result_2)
        return x.grad.data, v.grad.data, k.grad.data
    
    def gradient_stop_test(self, x, y, k):
        x = Variable(np.array([[1.,2.],[4.,5.]]))
        v = Variable(np.array([[4.,5.],[6.,7.]]))
        k = Variable(np.array([[1.,3.],[4.,6.]]))
        
        with GradientTape() as tape_1, GradientTape() as tape_2:
            result = ef.matmul(x,sf.transpose(v))
            result_2 = ef.matmul(ef.stop_gradient(result), k)
            result_2 = result_2 ** 2
            result_3 = result_2 * x
        tape_2.CalcGradient()
        return x.grad.data, k.grad.data
    
class LinalgFunction():
    def linalg_concat_test(self):
        x = Variable(np.array([[[3.0,2.0,1.0]],[[6.,5.,4.]]]))
        y = Variable(np.array([[[1.0,2.0,3.0]],[[4.,5.,6.]]]))
        k = Variable(np.array([[[1.0,2.0,5.0]],[[7.,8.,9.]]]))

        with GradientTape() as tape:
            d = x * y
            concats = flowconcat([k,d])
            result = concats * concats

        tape.CalcGradient()

        return concats.grad.data, k.grad.data, d.grad.data, x.grad.data, y.grad.data
    
    def linalg_stack_test(self):
        x = Variable(np.array([[[3.0,2.0,1.0]],[[6.,5.,4.]]]))
        y = Variable(np.array([[[1.0,2.0,3.0]],[[4.,5.,6.]]]))
        k = Variable(np.array([[[1.0,2.0,5.0]],[[7.,8.,9.]]]))
       
        with GradientTape() as tape:
            d = x * y
            concats = flowstack([k,d])
            result = concats * concats

        tape.CalcGradient()
        
        return concats.grad.data, k.grad.data, d.grad.data, x.grad.data, y.grad.data

In [342]:
class UnitTest():
    def __init__(self):
        self.dataset_path = tf.keras.utils.get_file(
            "auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
        self.jacobian_preset = JacobianFunction()
        self.function_preset = TestFunction()
        self.order_preset = OrderFunction()
        self.start_preset = GradientStartFunction()
        self.answer_preset = TestAnswer()
        self.linalg_preset = LinalgFunction()
        
    def data_preprocessing(self):
        def norm(x):
            return (x - train_stats['mean']) / train_stats['std']
        column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                        'Acceleration', 'Model Year', 'Origin']
        raw_dataset = pd.read_csv(self.dataset_path, names=column_names,
                              na_values = "?", comment='\t',
                              sep=" ", skipinitialspace=True)

        dataset = raw_dataset.copy()
        dataset = dataset.dropna()
        origin = dataset.pop('Origin')
        
        dataset['USA'] = (origin == 1)*1.0
        dataset['Europe'] = (origin == 2)*1.0
        dataset['Japan'] = (origin == 3)*1.0
        
        train_dataset = dataset.sample(frac=0.8,random_state=0)
        test_dataset = dataset.drop(train_dataset.index)
        
        train_stats = train_dataset.describe()
        train_stats.pop("MPG")
        train_stats = train_stats.transpose()
        
        normed_train_data = norm(train_dataset)
        normed_test_data = norm(test_dataset)
        train_labels = train_dataset.pop('MPG')
        
        X = Variable(np.array(normed_train_data.drop('MPG',axis = 1)))
        y = (np.array(train_labels) - np.mean(np.array(train_dataset))) / np.std(np.array(train_dataset))
        y = Variable(np.array(y).reshape([-1,1]))
        
        return X,y
    
    def answer_correction(self, function, answer, pred, tor):
        answer = [np.abs(i.reshape(j.shape)) for i,j in zip(answer, pred)]
        pred = [np.abs(x) for x in pred]
        loss = [np.abs(i - j) for i,j in zip(answer, pred)]
        loss = np.array([i.sum() for i in loss])
        if np.all(loss < tor):
            print(function, ' : ok')
        else :
            print(function, ' : Failed')
            print('answer :', answer)
            print('pred :', pred)
            print('error :', loss)

    def preset_test(self, tor = 0.01, function = None, x = None, y = None):
        try: 
            if x is None:
                x = Variable(np.array(1.0))
            if y is None:
                y = Variable(np.array(1.0))
            function_list = [i for i in self.function_preset.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = self.function_preset.__getattribute__(function)
                with GradientTape() as tape:
                    z = current_function(x, y)
                tape.CalcGradient()
                pred = (x.grad.data, y.grad.data)
                answer = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answer, pred, tor)
                tape.resetgrads()
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e)
            
    def high_order_test(self, orders = 2):
        try:        
            x = Variable(np.array(2.0))
            y = Variable(np.array(2.0))
            order_function = self.order_preset.high_order_function
            tape_dict = dict()
            with GradientTape() as tape:
                f = order_function(x, y)
            tape_dict[0] = tape
            for order in range(orders):
                with GradientTape() as tape_1:
                    tape_dict[order].CalcGradient()
                tape_dict[order + 1] = tape_1
                pred = (x.grad.data, y.grad.data)
                tape_dict[order].resetgrads()
                answer = self.answer_preset.high_order_function(order)
                self.answer_correction(f'high_order_test_{order}', answer, pred, 0.01)
                tape.resetgrads()
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e)
            
    def matrix_test(self, orders = 3):
        try:
            X = Variable(np.array([[1.,2.],[4.,5.]]))
            matrix_function = self.order_preset.matrix_order_function
            tape_dict = dict()
            with GradientTape() as tape:
                result = matrix_function(X)
            tape_dict[0] = tape
            for order in range(orders):    
                with GradientTape() as tape_1:
                    tape_dict[order].CalcGradient()
                tape_dict[order + 1] = tape_1
                pred = X.grad.data
                tape_dict[order].resetgrads()
                answer = self.answer_preset.matrix_order_function(order)
                self.answer_correction(f'matrix_order_test_{order}',
                                      answer,
                                      pred,
                                      0.01)
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e)
            
    def jacobian_test(self, tor = 0.01, x = None, v = None, k = None, target = None):
        try :
            if x is None:
                x = Variable(np.array([[1.,2.],[4.,5.]]))
            if v is None:
                v = Variable(np.array([[4.,5.],[6.,7.]]))
            if k is None:
                k = Variable(np.array([[1.,3.],[4.,6.]]))
            jacobian_function = self.jacobian_preset
            function_list = [i for i in jacobian_function.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = jacobian_function.__getattribute__(function)
                tape = current_function(x,v,k)
                if target is not None:
                    pred = (tape.jacobian(target = target, var = x, var_return = 'numpy'),
                     tape.jacobian(target = target, var = v, var_return = 'numpy'),
                     tape.jacobian(target = target, var = k, var_return = 'numpy'))
                else :
                    pred = (tape.jacobian(var = x, var_return = 'numpy'),
                     tape.jacobian(var = v, var_return = 'numpy'),
                     tape.jacobian(var = k, var_return = 'numpy'))
                answer = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answer, pred, tor)
                tape.resetgrads()
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e) 

    def gradient_start_index_test(self, tor = 0.01, x = None, v = None, k = None):
        try :
            if x is None:
                x = Variable(np.array([[1.,2.],[4.,5.]]))
            if v is None:
                v = Variable(np.array([[4.,5.],[6.,7.]]))
            if k is None:
                k = Variable(np.array([[1.,3.],[4.,6.]]))
            start_index_function = self.start_preset
            function_list = [i for i in start_index_function.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = start_index_function.__getattribute__(function)
                pred = current_function(x, v, k)
                answer = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answer, pred, tor)
        except Exception as e:
            print(f'{function} is failed :', e) 
            
    def linalg_test(self, tor = 0.01):
        try:
            linalg_function = self.linalg_preset
            function_list = [i for i in linalg_function.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = linalg_function.__getattribute__(function)
                preds = current_function()
                answers = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answers, preds, tor)
        except Exception as e:
            print(f'{function} is failed :', e) 
            
    def modeling_test(self, tor = 1e-7, end_iter = 4):
        try:
            X,y = self.data_preprocessing()

            lr = 0.1
            loss_flow = []
            loss_iter = 0

            start_time = time.time()
            model = Models(100, 1)
            optimizers = optimizer.Adam()
            optimizers.setup(model)
            for i in range(10000):
                with GradientTape() as tape:
                    y_pred = model(X)

                    loss = f.loss.mean_squared_error(y, y_pred)

                tape.CalcGradient()

                optimizers.update()

                if i % 1000 == 0:
                    loss_flow.append(loss.data)
                    if (loss_iter > end_iter):
                        if (abs(loss_flow[loss_iter - 1]) - (
                            abs(loss_flow[loss_iter]))) < tor:
                            print('loss_value :' , [float(i) for i in loss_flow])
                            print('model is under local minima, Attempt to retry..')
                            self.modeling_test()
                            break
                        else :
                            print('loss_value :' , [float(i) for i in loss_flow])
                            print('model training test : ok')
                            break
                    loss_iter += 1
        except Exception as e:
            print(f'model_training test is failed : {e}')
    def start_testing(self):
        self.high_order_test()
        self.matrix_test()
        self.jacobian_test()
        self.gradient_start_index_test()
        self.modeling_test()
        self.linalg_test()

In [343]:
class TestAnswer():
    
    def matyas(self, x = None, y = None):
        if (x is None) | (y is None):
            return (0.040000000000000036, 0.040000000000000036)
        else :
            return

    def goldstein(self, x = None, y = None):
        if (x is None) | (y is None):
            return (-5376.0, 8064.0)
        else :
            return
        
    def high_order_function(self, order, x = None, y = None):        
        if order == 0:
            if (x is None) | (y is None):
                return 36., 12.3535
            else :
                return
            
        if order == 1:
            if (x is None) | (y is None):
                return 50, 11.911
            else :
                return
    
        else :
            return
    
    def matrix_order_function(self, order, x = None, y = None):
        if order == 0:
            if (x is None) | (y is None):
                return np.array([[  6.,  24.], [ 96., 150.]])
            else :
                return
            
        if order == 1:
            if (x is None) | (y is None):
                return np.array([[12., 24.], [48., 60.]])
            else :
                return
    
        if order == 2:
            if (x is None) | (y is None):
                return np.array([[12., 12.], [12., 12.]])
            else :
                return
            
    def matmul(self, x = None, y = None, k = None):
        answer = []
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            for i in ['x','y','k']:
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)

                answer.append(tape.jacobian(result_2, eval(i)).numpy())
            return tuple(answer)
        
    def reduce_sum(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']:        
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = tf.math.reduce_sum(result_2)

                answer.append(tape.jacobian(result_3, eval(i)).numpy())
            
            return tuple(answer)
        
    def div(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']: 
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = result_2 / result

                answer.append(tape.jacobian(result_3, eval(i)).numpy())

            return tuple(answer)
    
    def sum(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']:             
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = result_2 + result

                answer.append(tape.jacobian(result_3, eval(i)).numpy())
            
            return tuple(answer)
        
    def mul(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']:             
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = result_2 * result

                answer.append(tape.jacobian(result_3, eval(i)).numpy())
            
            return tuple(answer)
        
    def start_middle(self, x = None, y = None, k = None):
        x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
        v = tf.Variable(np.array([[4.,5.],[6.,7.]]))
        k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
        
        with tf.GradientTape() as tape:
            result = tf.matmul(x,tf.transpose(v))
            result_2 = tf.matmul(result, k)
            result_3 = result_2 / x
            
        answer = tape.gradient(result_2, [x, v])
        
        return answer
    
    def gradient_start_middle(self, x = None, y = None, k = None):
        x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
        v = tf.Variable(np.array([[4.,5.],[6.,7.]]))
        k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
        
        with tf.GradientTape() as tape:
            result = tf.matmul(x,tf.transpose(v))
            result_2 = tf.matmul(result, k)
            result_3 = result_2 / x
            
        answer = tape.gradient(result_2, [x, v, k])
        answer = [i.numpy() for i in answer]
        
        return answer
    
    def gradient_stop_test(self, x = None, y = None, k = None):
        x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
        v = tf.Variable(np.array([[4.,5.],[6.,7.]]))
        k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
        
        with tf.GradientTape() as tape:
            result = tf.matmul(x,tf.transpose(v))
            result_2 = tf.matmul(tf.stop_gradient(result), k)
            result_2 = result_2 ** 2
            result_3 = result_2 * x
            
        answer = tape.gradient(result_3, [x, k])
        answer = [i.numpy() for i in answer]
        
        return answer
    
    def linalg_concat_test(self):
        a = np.array([[[ 2.,  4., 10.]],
                      [[14., 16., 18.]],
                      [[ 6.,  8.,  6.]],
                      [[48., 50., 48.]]])

        b = np.array([[[ 2.,  4., 10.]],
                      [[14., 16., 18.]]])

        c = np.array([[[ 6.,  8.,  6.]],
                      [[48., 50., 48.]]])

        d = np.array([[[  6.,  16.,  18.]],
                      [[192., 250., 288.]]])

        e = np.array([[[ 18.,  16.,   6.]],
                      [[288., 250., 192.]]])

        return a,b,c,d,e
        
    def linalg_stack_test(self):
        a = np.array([[[[ 2.,  4., 10.]],
                       [[14., 16., 18.]]],
                      [[[ 6.,  8.,  6.]],
                       [[48., 50., 48.]]]])

        b = np.array([[[ 2.,  4., 10.]],
                      [[14., 16., 18.]]])

        c = np.array([[[ 6.,  8.,  6.]],
                      [[48., 50., 48.]]])

        d = np.array([[[  6.,  16.,  18.]],
                      [[192., 250., 288.]]])

        e = np.array([[[ 18.,  16.,   6.]],
                      [[288., 250., 192.]]])
            
        return a,b,c,d,e

In [344]:
class Models(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l2 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l3 = layer.Linear(out_size, initializer_func='he_uniform')
        
    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(y)
        y = f.activation.relu(y)
        y = self.l3(y)
        return y

In [345]:
test = UnitTest()

In [346]:
test.linalg_test()

linalg_concat_test  : ok
linalg_stack_test  : ok


In [255]:
[i.sum() for i in test.test.tolist()]

AttributeError: 'UnitTest' object has no attribute 'test'

In [18]:
test.start_testing()

high_order_test_0  : ok
high_order_test_1  : ok
matrix_order_test_0  : ok
matrix_order_test_1  : ok
matrix_order_test_2  : ok


NameError: name 'order' is not defined

In [169]:
test.high_order_test()

high_order_test_0  : ok
high_order_test_1  : ok


In [170]:
test.matrix_test()

matrix_order_test_0  : ok
matrix_order_test_1  : ok
matrix_order_test_2  : ok


In [171]:
test.jacobian_test()

matmul  : ok
reduce_sum  : ok
div  : ok
sum  : ok
mul  : ok


In [172]:
test.gradient_start_index_test()

gradient_start_middle  : ok
gradient_stop_test  : ok


In [173]:
test.modeling_test()

loss_value : [array(0.12923438), array(4.02457493e-06), array(3.32494054e-06), array(3.18126507e-06), array(2.89203548e-06), array(2.65531922e-06)]
model training test : ok


In [67]:
class Models(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l2 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l3 = layer.Linear(out_size, initializer_func='he_uniform')
        
    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(y)
        y = f.activation.relu(y)
        y = self.l3(y)
        return y

In [68]:
model = Models(100, 1)

optimizers = optimizer.Adam()

optimizers.setup(model)

In [69]:
start_time = time.time()
for i in range(10000):
    with GradientTape() as tape:
        y_pred = model(X)
        
        loss = f.loss.mean_squared_error(y, y_pred)
        
    tape.CalcGradient()
    
    optimizers.update()
    
    if i % 1000 == 0:
        print(loss.data)

print('total_time : ', time.time() - start_time)

0.13316079589785426
4.678867569526384e-06
3.323818075624142e-06
2.434985858024435e-06
1.7888589763894105e-06
1.3856699026652211e-06
1.1175656879502677e-06
9.268755179942446e-07
8.51791997634991e-07
6.456100152307053e-07
total_time :  12.611826419830322


In [318]:
a = np.array([1,2])

In [24]:
b = cp.asarray(a)

In [39]:
a_cp = cp.get_array_module(a.data)

In [42]:
a_cp.array(a.data)

array([[1, 3],
       [4, 5],
       [6, 8]])

In [20]:
cp.arange(5).reshape(2,3)

ValueError: cannot reshape array of size 5 into shape (2, 3)

In [ ]:
# =============================================================================
# [simple version] conv2d_simple / pooling_simple
# =============================================================================
def conv2d_simple(x, W, b=None, stride=1, pad=0):
    x, W = as_variable(x), as_variable(W)

    Weight = W
    N, C, H, W = x.shape
    OC, C, KH, KW = Weight.shape
    SH, SW = pair(stride)
    PH, PW = pair(pad)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    col = im2col(x, (KH, KW), stride, pad, to_matrix=True)
    Weight = Weight.reshape(OC, -1).transpose()
    t = linear(col, Weight, b)
    y = t.reshape(N, OH, OW, OC).transpose(0, 3, 1, 2)
    return y


def pooling_simple(x, kernel_size, stride=1, pad=0):
    x = as_variable(x)

    N, C, H, W = x.shape
    KH, KW = pair(kernel_size)
    PH, PW = pair(pad)
    SH, SW = pair(stride)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    col = im2col(x, kernel_size, stride, pad, to_matrix=True)
    col = col.reshape(-1, KH * KW)
    y = col.max(axis=1)
    y = y.reshape(N, OH, OW, C).transpose(0, 3, 1, 2)
    return y


In [ ]:
import numpy as np
from dezero import cuda
from dezero.core import Function, as_variable
from dezero.utils import pair, get_conv_outsize, get_deconv_outsize
from dezero.functions import linear, broadcast_to




# =============================================================================
#  conv2d / deconv2d
# =============================================================================
class Conv2d(Function):
    def __init__(self, stride=1, pad=0):
        super().__init__()
        self.stride = pair(stride)
        self.pad = pair(pad)

    def forward(self, x, W, b):
        xp = cuda.get_array_module(x)

        KH, KW = W.shape[2:]
        col = im2col_array(x, (KH, KW), self.stride, self.pad, to_matrix=False)

        y = xp.tensordot(col, W, ((1, 2, 3), (1, 2, 3)))
        if b is not None:
            y += b
        y = xp.rollaxis(y, 3, 1)
        # y = np.transpose(y, (0, 3, 1, 2))
        return y

    def backward(self, gy):
        x, W, b = self.inputs
        # ==== gx ====
        gx = deconv2d(gy, W, b=None, stride=self.stride, pad=self.pad,
                      outsize=(x.shape[2], x.shape[3]))
        # ==== gW ====
        gW = Conv2DGradW(self)(x, gy)
        # ==== gb ====
        gb = None
        if b.data is not None:
            gb = gy.sum(axis=(0, 2, 3))
        return gx, gW, gb


def conv2d(x, W, b=None, stride=1, pad=0):
    return Conv2d(stride, pad)(x, W, b)


class Deconv2d(Function):
    def __init__(self, stride=1, pad=0, outsize=None):
        super().__init__()
        self.stride = pair(stride)
        self.pad = pair(pad)
        self.outsize = outsize

    def forward(self, x, W, b):
        xp = cuda.get_array_module(x)

        Weight = W
        SH, SW = self.stride
        PH, PW = self.pad
        C, OC, KH, KW = Weight.shape
        N, C, H, W = x.shape
        if self.outsize is None:
            out_h = get_deconv_outsize(H, KH, SH, PH)
            out_w = get_deconv_outsize(W, KW, SW, PW)
        else:
            out_h, out_w = pair(self.outsize)
        img_shape = (N, OC, out_h, out_w)

        gcol = xp.tensordot(Weight, x, (0, 1))
        gcol = xp.rollaxis(gcol, 3)
        y = col2im_array(gcol, img_shape, (KH, KW), self.stride, self.pad,
                         to_matrix=False)
        # b, k, h, w
        if b is not None:
            self.no_bias = True
            y += b.reshape((1, b.size, 1, 1))
        return y

    def backward(self, gy):
        x, W, b = self.inputs

        # ==== gx ====
        gx = conv2d(gy, W, b=None, stride=self.stride, pad=self.pad)
        # ==== gW ====
        f = Conv2DGradW(self)
        gW = f(gy, x)
        # ==== gb ====
        gb = None
        if b.data is not None:
            gb = gy.sum(axis=(0, 2, 3))
        return gx, gW, gb


def deconv2d(x, W, b=None, stride=1, pad=0, outsize=None):
    return Deconv2d(stride, pad, outsize)(x, W, b)


class Conv2DGradW(Function):
    def __init__(self, conv2d):
        W = conv2d.inputs[1]
        kh, kw = W.shape[2:]
        self.kernel_size = (kh, kw)
        self.stride = conv2d.stride
        self.pad = conv2d.pad

    def forward(self, x, gy):
        xp = cuda.get_array_module(x)

        col = im2col_array(x, self.kernel_size, self.stride, self.pad,
                           to_matrix=False)
        gW = xp.tensordot(gy, col, ((0, 2, 3), (0, 4, 5)))
        return gW

    def backward(self, gys):
        x, gy = self.inputs
        gW, = self.outputs

        xh, xw = x.shape[2:]
        gx = deconv2d(gy, gW, stride=self.stride, pad=self.pad,
                      outsize=(xh, xw))
        ggy = conv2d(x, gW, stride=self.stride, pad=self.pad)
        return gx, ggy


# =============================================================================
#  pooling(max-pooling) / average_pooling
# =============================================================================
class Pooling(Function):
    def __init__(self, kernel_size, stride=1, pad=0):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        col = im2col_array(x, self.kernel_size, self.stride, self.pad,
                           to_matrix=False)

        N, C, KH, KW, OH, OW = col.shape
        col = col.reshape(N, C, KH * KW, OH, OW)
        self.indexes = col.argmax(axis=2)
        y = col.max(axis=2)
        return y

    def backward(self, gy):
        return Pooling2DGrad(self)(gy)


class Pooling2DGrad(Function):
    def __init__(self, mpool2d):
        self.mpool2d = mpool2d
        self.kernel_size = mpool2d.kernel_size
        self.stride = mpool2d.stride
        self.pad = mpool2d.pad
        self.input_shape = mpool2d.inputs[0].shape
        self.dtype = mpool2d.inputs[0].dtype
        self.indexes = mpool2d.indexes

    def forward(self, gy):
        xp = cuda.get_array_module(gy)

        N, C, OH, OW = gy.shape
        N, C, H, W = self.input_shape
        KH, KW = pair(self.kernel_size)

        gcol = xp.zeros((N * C * OH * OW * KH * KW), dtype=self.dtype)

        indexes = (self.indexes.ravel()
                   + xp.arange(0, self.indexes.size * KH * KW, KH * KW))
        
        gcol[indexes] = gy.ravel()
        gcol = gcol.reshape(N, C, OH, OW, KH, KW)
        gcol = xp.swapaxes(gcol, 2, 4)
        gcol = xp.swapaxes(gcol, 3, 5)

        gx = col2im_array(gcol, (N, C, H, W), self.kernel_size, self.stride,
                          self.pad, to_matrix=False)
        return gx

    def backward(self, ggx):
        f = Pooling2DWithIndexes(self.mpool2d)
        return f(ggx)


class Pooling2DWithIndexes(Function):
    def __init__(self, mpool2d):
        self.kernel_size = mpool2d.kernel_size
        self.stride = mpool2d.stride
        self.pad = mpool2d.pad
        self.input_shpae = mpool2d.inputs[0].shape
        self.dtype = mpool2d.inputs[0].dtype
        self.indexes = mpool2d.indexes

    def forward(self, x):
        col = im2col_array(x, self.kernel_size, self.stride, self.pad,
                           to_matrix=False)
        N, C, KH, KW, OH, OW = col.shape
        col = col.reshape(N, C, KH * KW, OH, OW)
        col = col.transpose(0, 1, 3, 4, 2).reshape(-1, KH * KW)
        indexes = self.indexes.ravel()
        col = col[np.arange(len(indexes)), indexes]
        return col.reshape(N, C, OH, OW)


def pooling(x, kernel_size, stride=1, pad=0):
    return Pooling(kernel_size, stride, pad)(x)


class AveragePooling(Function):
    def __init__(self, kernel_size, stride=1, pad=0):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = pad
        self.input_shape = None

    def forward(self, x):
        self.input_shape = x.shape
        col = im2col_array(x, self.kernel_size, self.stride, self.pad,
                           to_matrix=False)
        y = col.mean(axis=(2, 3))
        return y

    def backward(self, gy):
        # TODO(Koki): This is simple implementation
        N, C, OH, OW = gy.shape
        KW, KH = pair(self.kernel_size)
        gy /= (KW*KH)
        gcol = broadcast_to(gy.reshape(-1), (KH, KW, N*C*OH*OW))
        gcol = gcol.reshape(KH, KW, N, C, OH, OW).transpose(2, 3, 0, 1, 4, 5)
        gx = col2im(gcol, self.input_shape, self.kernel_size, self.stride,
                    self.pad, to_matrix=False)
        return gx


def average_pooling(x, kernel_size, stride=1, pad=0):
    return AveragePooling(kernel_size, stride, pad)(x)


# =============================================================================
#  im2col / col2im
# =============================================================================
class Im2col(Function):
    def __init__(self, kernel_size, stride, pad, to_matrix):
        super().__init__()
        self.input_shape = None
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = pad
        self.to_matrix = to_matrix

    def forward(self, x):
        self.input_shape = x.shape
        y = im2col_array(x, self.kernel_size, self.stride, self.pad,
                         self.to_matrix)
        return y

    def backward(self, gy):
        gx = col2im(gy, self.input_shape, self.kernel_size, self.stride,
                    self.pad, self.to_matrix)
        return gx


def im2col(x, kernel_size, stride=1, pad=0, to_matrix=True):
    """Extract patches from an image based on the filter.
    Args:
        x (`dezero.Variable` or `ndarray`): Input variable of shape
            `(N, C, H, W)`
        kernel_size (int or (int, int)): Size of kernel.
        stride (int or (int, int)): Stride of kernel.
        pad (int or (int, int)): Spatial padding width for input arrays.
        to_matrix (bool): If True the `col` will be reshaped to 2d array whose
            shape is `(N*OH*OW, C*KH*KW)`
    Returns:
        `dezero.Variable`: Output variable. If the `to_matrix` is False, the
            output shape is `(N, C, KH, KW, OH, OW)`, otherwise
            `(N*OH*OW, C*KH*KW)`.
    Notation:
    - `N` is the batch size.
    - `C` is the number of the input channels.
    - `H` and `W` are the height and width of the input image, respectively.
    - `KH` and `KW` are the height and width of the filters, respectively.
    - `SH` and `SW` are the strides of the filter.
    - `PH` and `PW` are the spatial padding sizes.
    - `OH` and `OW` are the the height and width of the output, respectively.
    """
    y = Im2col(kernel_size, stride, pad, to_matrix)(x)
    return y


class Col2im(Function):
    def __init__(self, input_shape, kernel_size, stride, pad, to_matrix):
        super().__init__()
        self.input_shape = input_shape
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = pad
        self.to_matrix = to_matrix

    def forward(self, x):
        y = col2im_array(x, self.input_shape, self.kernel_size, self.stride,
                         self.pad, self.to_matrix)
        return y

    def backward(self, gy):
        gx = im2col(gy, self.kernel_size, self.stride, self.pad,
                    self.to_matrix)
        return gx


def col2im(x, input_shape, kernel_size, stride=1, pad=0, to_matrix=True):
    return Col2im(input_shape, kernel_size, stride, pad, to_matrix)(x)


# =============================================================================
#  numpy im2col
# =============================================================================
def im2col_array(img, kernel_size, stride, pad, to_matrix=True):

    N, C, H, W = img.shape
    KH, KW = pair(kernel_size)
    SH, SW = pair(stride)
    PH, PW = pair(pad)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    xp = cuda.get_array_module(img)
    if xp != np:
        col = _im2col_gpu(img, kernel_size, stride, pad)
    else:
        img = np.pad(img,
                     ((0, 0), (0, 0), (PH, PH + SH - 1), (PW, PW + SW - 1)),
                     mode='constant', constant_values=(0,))
        col = np.ndarray((N, C, KH, KW, OH, OW), dtype=img.dtype)

        for j in range(KH):
            j_lim = j + SH * OH
            for i in range(KW):
                i_lim = i + SW * OW
                col[:, :, j, i, :, :] = img[:, :, j:j_lim:SH, i:i_lim:SW]

    if to_matrix:
        col = col.transpose((0, 4, 5, 1, 2, 3)).reshape((N * OH * OW, -1))

    return col


def col2im_array(col, img_shape, kernel_size, stride, pad, to_matrix=True):
    N, C, H, W = img_shape
    KH, KW = pair(kernel_size)
    SH, SW = pair(stride)
    PH, PW = pair(pad)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    if to_matrix:
        col = col.reshape(N, OH, OW, C, KH, KW).transpose(0, 3, 4, 5, 1, 2)

    xp = cuda.get_array_module(col)
    if xp != np:
        img = _col2im_gpu(col, SH, SW, PH, PW, H, W)
        return img
    else:
        img = np.zeros((N, C, H + 2 * PH + SH - 1, W + 2 * PW + SW - 1),
                       dtype=col.dtype)
        for j in range(KH):
            j_lim = j + SH * OH
            for i in range(KW):
                i_lim = i + SW * OW
                img[:, :, j:j_lim:SH, i:i_lim:SW] += col[:, :, j, i, :, :]
        return img[:, :, PH:H + PH, PW:W + PW]


def _im2col_gpu(img, kernel_size, stride, pad):
    """im2col function for GPU.
    This code is ported from Chainer:
    https://github.com/chainer/chainer/blob/v6.4.0/chainer/utils/conv.py
    """
    n, c, h, w = img.shape
    kh, kw = pair(kernel_size)
    sy, sx = pair(stride)
    ph, pw = pair(pad)
    out_h = get_conv_outsize(h, kh, sy, ph)
    out_w = get_conv_outsize(w, kw, sx, pw)
    dy, dx = 1, 1
    col = cuda.cupy.empty((n, c, kh, kw, out_h, out_w), dtype=img.dtype)

    cuda.cupy.ElementwiseKernel(
        'raw T img, int32 h, int32 w, int32 out_h, int32 out_w,'
        'int32 kh, int32 kw, int32 sy, int32 sx, int32 ph, int32 pw,'
        'int32 dy, int32 dx',
        'T col',
        '''
           int c0 = i / (kh * kw * out_h * out_w);
           int ky = i / (kw * out_h * out_w) % kh;
           int kx = i / (out_h * out_w) % kw;
           int out_y = i / out_w % out_h;
           int out_x = i % out_w;
           int in_y = ky * dy + out_y * sy - ph;
           int in_x = kx * dx + out_x * sx - pw;
           if (in_y >= 0 && in_y < h && in_x >= 0 && in_x < w) {
             col = img[in_x + w * (in_y + h * c0)];
           } else {
             col = 0;
           }
        ''',
        'im2col')(img.reduced_view(),
                  h, w, out_h, out_w, kh, kw, sy, sx, ph, pw, dy, dx, col)

    return col


def _col2im_gpu(col, sy, sx, ph, pw, h, w):
    """col2im function for GPU.
    This code is ported from Chainer:
    https://github.com/chainer/chainer/blob/v6.4.0/chainer/utils/conv.py
    """
    n, c, kh, kw, out_h, out_w = col.shape
    dx, dy = 1, 1
    img = cuda.cupy.empty((n, c, h, w), dtype=col.dtype)

    cuda.cupy.ElementwiseKernel(
        'raw T col, int32 h, int32 w, int32 out_h, int32 out_w,'
        'int32 kh, int32 kw, int32 sy, int32 sx, int32 ph, int32 pw,'
        'int32 dx, int32 dy',
        'T img',
        '''
           int c0 = i / (h * w);
           int y  = i / w % h;
           int x  = i % w;
           T val = 0;
           for (int ky = 0; ky < kh; ++ky) {
             int out_y = (y + ph - ky * dy);
             if (0 > out_y || out_y >= out_h * sy) continue;
             if (out_y % sy != 0) continue;
             out_y /= sy;
             for (int kx = 0; kx < kw; ++kx) {
               int out_x = (x + pw - kx * dx);
               if (0 > out_x || out_x >= out_w * sx) continue;
               if (out_x % sx != 0) continue;
               out_x /= sx;
               int k = out_y + out_h * (kx + kw * (ky + kh * c0));
               val = val + col[out_x + out_w * k];
             }
           }
           img = val;
        ''',
        'col2im')(col.reduced_view(),
                  h, w, out_h, out_w, kh, kw, sy, sx, ph, pw, dx, dy, img)
    return img